In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import heapq
import wandb
plt.switch_backend('agg')
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 30

In [3]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.char2index = {}
        self.char2count = {}
        self.index2char = {0: "SOS", 1: "EOS"}
        self.n_chars = 2  # Count SOS and EOS

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1

In [4]:
def readLangs(lang, type, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    
    lines = open('aksharantar_sampled/%s/%s_%s.csv' % (lang, lang, type), encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    # pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]
    pairs = [[s for s in l.split(',')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang)
        output_lang = Lang('eng')
    else:
        input_lang = Lang('eng')
        output_lang = Lang(lang)

    return input_lang, output_lang, pairs

In [5]:
def prepareData(lang, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang, reverse)
    print("Read %s word pairs" % len(pairs))
    print("Trimmed to %s word pairs" % len(pairs))
    print("Counting chars...")
    for pair in pairs:
        input_lang.addWord(pair[0])
        output_lang.addWord(pair[1])
    print("Counted chars:")
    print(input_lang.name, input_lang.n_chars)
    print(output_lang.name, output_lang.n_chars)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('mni', 'train')
print(random.choice(pairs))

Reading lines...
Read 10060 word pairs
Trimmed to 10060 word pairs
Counting chars...
Counted chars:
eng 28
mni 46
['darini', 'ꯗꯥꯔꯤꯅꯤ']


In [6]:
cells = {'rnn': nn.RNN, 'gru': nn.GRU, 'lstm': nn.LSTM}

In [7]:
class EncoderRNN(nn.Module):
    """
    Class for the encoder RNN.
    """
    def __init__(self, input_size, embedding_size, hidden_size, num_hidden_layers, dropout=0.2, cell_type='gru'):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.cell_type = cell_type
        self.cell = cells[cell_type](embedding_size, hidden_size, num_hidden_layers, dropout=dropout)

    def forward(self, input, hidden, cell=None):
        embedded = self.embedding(input).view(1, 1, -1)
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            output, hidden = self.cell(embedded, hidden)
            return output, hidden
        else:
            output, (hidden, cell) = self.cell(embedded, (hidden, cell))    
            return output, hidden, cell

    def initHidden(self):
        return torch.zeros(self.num_hidden_layers, 1, self.hidden_size, device=device)

In [8]:
class BeamSearchNode:
    def __init__(self, decoder_output, hidden, prev_node, char_idx, log_prob, length):
        self.decoder_output = decoder_output
        self.hidden = hidden
        self.prev_node = prev_node
        self.char_idx = char_idx
        self.log_prob = log_prob
        self.length = length

    # def __lt__(self, other):
    #     return self.log_prob > other.log_prob

    def eval(self):
        return self.log_prob / self.length

In [9]:
class DecoderRNN(nn.Module):
    """
    Class for the vanilla decoder RNN.
    """
    def __init__(self, hidden_size, embedding_size, output_size, num_hidden_layers, dropout=0, cell_type='gru'):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.cell_type = cell_type
        self.cell = cells[cell_type](embedding_size, hidden_size, num_hidden_layers, dropout=dropout)
        self.out = nn.Linear(hidden_size, output_size)
        self.output_size = output_size
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell=None):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            output, hidden = self.cell(output, hidden)
        else:
            output, (hidden, cell) = self.cell(output, (hidden, cell))
            
        output = self.softmax(self.out(output[0]))

        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            return output, hidden
        else:
            return output, hidden, cell
    
    def initHidden(self):
        return torch.zeros(self.num_hidden_layers, 1, self.hidden_size, device=device)

    def beam_search(self, encoder_outputs, decoder_hidden, beam_width, max_length, start_token, end_token):
        # Initialize the beam search
        beam_nodes = [BeamSearchNode(None, decoder_hidden, None, start_token, 0, 1)]
        done_nodes = []

        # Keep expanding the beam until we reach the maximum length or all candidates are done
        for _ in range(max_length):
            candidates = []
            for node in beam_nodes:
                if node.char_idx == end_token:
                    done_nodes.append(node)
                    continue

                # Feed the previous char and hidden state into the decoder
                output, hidden = self.forward(torch.tensor([node.char_idx]), node.hidden)
                # Single depth or double depth for tensor?

                # Generate new candidate nodes and add them to the heap
                for char_idx in range(self.output_size):
                    length = node.length + 1
                    log_prob = node.log_prob + output[0][char_idx].item()
                    new_node = BeamSearchNode(output, hidden, node, char_idx, log_prob, length)
                    heapq.heappush(candidates, (-new_node.eval(), id(new_node), new_node))

            # Select the top k candidates to continue expanding the beam
            beam_nodes = []
            for _ in range(beam_width):
                if not candidates:
                    break
                _, _, node = heapq.heappop(candidates)
                beam_nodes.append(node)

            if not beam_nodes:
                break

        # Return the best candidate as the predicted sequence
        done_nodes.extend(beam_nodes)
        best_node = max(done_nodes, key=lambda node: node.eval())
        # predicted_seq = []
        # while best_node.prev_node is not None:
        #     predicted_seq.append(best_node.char_idx)
        #     best_node = best_node.prev_node
        # return predicted_seq[::-1]
        return best_node

In [10]:
def indexesFromWord(lang, word):
    return [lang.char2index[char] for char in word]


def tensorFromWord(lang, word):
    indexes = indexesFromWord(lang, word)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromWord(input_lang, pair[0])
    target_tensor = tensorFromWord(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, beam_width, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    if encoder.cell_type == 'lstm':
        encoder_cell = encoder.initHidden()

    loss = 0

    for ei in range(input_length):
        # print(cell_type)
        if encoder.cell_type == 'lstm':
            encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei], encoder_hidden, encoder_cell)
        else:
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_hidden = encoder_hidden

    if beam_width == 1:
        decoder_input = torch.tensor([[SOS_token]], device=device)

        if decoder.cell_type == 'lstm':
            decoder_cell = encoder_cell

        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            for di in range(target_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                # print("Criterion:", criterion(decoder_output, target_tensor[di])) 
                loss += criterion(decoder_output, target_tensor[di])
                # print(loss)
                decoder_input = target_tensor[di]  # Teacher forcing

        else:
            # Without teacher forcing: use its own predictions as the next input
            for di in range(target_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input

                # print("Criterion:", criterion(decoder_output, target_tensor[di])) 
                loss += criterion(decoder_output, target_tensor[di])
                # print(loss)
                if decoder_input.item() == EOS_token:
                    break

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length
    
    else:
        best_node = decoder.beam_search(encoder_outputs, decoder_hidden, beam_width, target_length, SOS_token, EOS_token)

        decoder_outputs = []
        curr_node = best_node

        while curr_node.decoder_output is not None:
            decoder_outputs.append(curr_node.decoder_output)
            curr_node = curr_node.prev_node

        decoder_outputs = decoder_outputs[::-1]

        # Loss is the sum of the criterion of each decoder output with respect to the target
        loss = sum([criterion(decoder_output, target_tensor[i]) for i, decoder_output in enumerate(decoder_outputs)])
        # print(loss)
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length


In [12]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainIters(encoder, decoder, n_iters, optimizer=optim.SGD, cell_type='gru', beam_width=1, print_every=1000, plot_every=100, learning_rate=5e-3):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optimizer(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optimizer(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for _ in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, cell_type, encoder_optimizer, decoder_optimizer, criterion, beam_width)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

In [14]:
def evaluate(encoder, decoder, word, beam_width=1, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromWord(input_lang, word)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        if encoder.cell_type == 'lstm':
            encoder_cell = encoder.initHidden()

        for ei in range(input_length):
            if encoder.cell_type == 'lstm':
                encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei], encoder_hidden, encoder_cell)
            else:
                encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
                
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_hidden = encoder_hidden

        if beam_width == 1:
            decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

            if encoder.cell_type == 'lstm':
                decoder_cell = encoder_cell

            decoded_chars = ""
            # decoder_attentions = torch.zeros(max_length, max_length)

            for di in range(max_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                    
                # decoder_attentions[di] = decoder_attention.data
                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token:
                    decoded_chars += '<EOS>'
                    break
                else:
                    decoded_chars += output_lang.index2char[topi.item()]

                decoder_input = topi.squeeze().detach()

            # return decoded_chars, decoder_attentions[:di + 1]
            return decoded_chars
        
        else:
            predicted_seq = decoder.beam_search(encoder_outputs, decoder_hidden, beam_width, SOS_token, EOS_token)
            return ''.join([output_lang.index2char[char_idx] for char_idx in predicted_seq])

In [15]:
input_valid_lang, output_valid_lang, valid_pairs = prepareData('mni', 'valid')
input_test_lang, output_test_lang, test_pairs = prepareData('mni', 'test')

def evalAccuracy(encoder, decoder, pairs):
    correct = 0
    for pair in pairs:
        output_word = evaluate(encoder, decoder, pair[0])
        output_word = output_word[:-5]
        if output_word == pair[1]:
            correct += 1
    return correct / len(pairs)

Reading lines...
Read 3260 word pairs
Trimmed to 3260 word pairs
Counting chars...
Counted chars:
eng 28
mni 46
Reading lines...
Read 4096 word pairs
Trimmed to 4096 word pairs
Counting chars...
Counted chars:
eng 28
mni 46


In [16]:
def trainBatch(encoder, decoder, epochs, optimizer=optim.SGD, beam_width=1, print_every=1000, learning_rate=5e-3, log=False):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optimizer(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optimizer(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(pair) for pair in pairs]
    criterion = nn.NLLLoss()

    # print("Training pairs:", len(training_pairs))

    for iter in range(1, len(training_pairs) * epochs + 1):
        training_pair = training_pairs[(iter - 1) % len(training_pairs)]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, beam_width)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / (len(training_pairs) * epochs)), iter, iter / (len(training_pairs) * epochs) * 100, print_loss_avg))

        if log:
            wandb.log({"train_loss": loss})

            if iter % len(training_pairs) == 0:
                wandb.log({"train_accuracy": evalAccuracy(encoder, decoder, pairs), "val_accuracy": evalAccuracy(encoder, decoder, valid_pairs), "epoch": iter / len(training_pairs)})

In [17]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        # output_chars, attentions = evaluate(encoder, decoder, pair[0])
        output_word = evaluate(encoder, decoder, pair[0])
        # output_word = ''.join(output_chars)
        print('<', output_word)
        print('')

In [22]:
hidden_size = 256
# embedding_size = input_lang.n_chars
embedding_size = 256
encoder1 = EncoderRNN(input_lang.n_chars, embedding_size, hidden_size, num_hidden_layers=1, cell_type='gru').to(device)
decoder1 = DecoderRNN(hidden_size, embedding_size, output_lang.n_chars, num_hidden_layers=1, cell_type='gru').to(device)

/home/abdullah/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [23]:
# trainIters(encoder1, decoder1, n_iters=1, beam_width=5, print_every=100)

In [24]:
trainBatch(encoder1, decoder1, epochs=4, beam_width=1, print_every=100)

0m 4s (- 32m 55s) (100 0%) 2.8375
0m 9s (- 32m 53s) (200 0%) 2.8439
0m 12s (- 28m 48s) (300 0%) 2.8731
0m 16s (- 26m 35s) (400 0%) 2.8074
0m 18s (- 24m 25s) (500 1%) 2.8058
0m 20s (- 22m 57s) (600 1%) 2.7203
0m 22s (- 21m 32s) (700 1%) 2.7708
0m 25s (- 20m 50s) (800 1%) 2.5993
0m 29s (- 21m 14s) (900 2%) 2.6548
0m 32s (- 21m 14s) (1000 2%) 2.5260
0m 34s (- 20m 26s) (1100 2%) 2.7204
0m 36s (- 19m 54s) (1200 2%) 2.5600
0m 38s (- 19m 21s) (1300 3%) 2.5241
0m 41s (- 19m 8s) (1400 3%) 2.5929
0m 43s (- 18m 46s) (1500 3%) 2.6306
0m 45s (- 18m 30s) (1600 3%) 2.5470
0m 49s (- 18m 45s) (1700 4%) 2.5384
0m 52s (- 18m 50s) (1800 4%) 2.6067
0m 55s (- 18m 38s) (1900 4%) 2.6096
0m 57s (- 18m 22s) (2000 4%) 2.6454
1m 0s (- 18m 13s) (2100 5%) 2.5470
1m 3s (- 18m 11s) (2200 5%) 2.5591
1m 5s (- 18m 8s) (2300 5%) 2.5236
1m 8s (- 18m 2s) (2400 5%) 2.5447
1m 10s (- 17m 46s) (2500 6%) 2.5981
1m 13s (- 17m 42s) (2600 6%) 2.5934
1m 15s (- 17m 34s) (2700 6%) 2.6466
1m 18s (- 17m 29s) (2800 6%) 2.5177
1m 21s (- 

In [25]:
encoder2 = EncoderRNN(input_lang.n_chars, embedding_size, hidden_size, num_hidden_layers=2, cell_type='gru').to(device)
decoder2 = DecoderRNN(hidden_size, embedding_size, output_lang.n_chars, num_hidden_layers=2, cell_type='gru').to(device)

In [26]:
trainBatch(encoder2, decoder2, epochs=4, beam_width=5, print_every=100)

0m 8s (- 58m 20s) (100 0%) 2.0147
0m 17s (- 57m 29s) (200 0%) 2.0716
0m 25s (- 57m 20s) (300 0%) 2.2560
0m 34s (- 57m 31s) (400 0%) 2.3365
0m 43s (- 57m 3s) (500 1%) 2.5533
0m 50s (- 55m 52s) (600 1%) 2.4879
0m 58s (- 55m 32s) (700 1%) 2.5480
1m 6s (- 54m 57s) (800 1%) 2.5758
1m 14s (- 54m 11s) (900 2%) 2.4195
1m 22s (- 53m 45s) (1000 2%) 2.6934
1m 32s (- 54m 59s) (1100 2%) 2.3984
1m 39s (- 54m 7s) (1200 2%) 2.4747
1m 51s (- 55m 34s) (1300 3%) 2.3887
2m 1s (- 56m 20s) (1400 3%) 2.3587
2m 10s (- 56m 23s) (1500 3%) 2.5168
2m 20s (- 56m 28s) (1600 3%) 2.6045
2m 29s (- 56m 37s) (1700 4%) 2.6799
2m 40s (- 57m 1s) (1800 4%) 2.7811
2m 45s (- 55m 43s) (1900 4%) 2.7165
2m 51s (- 54m 44s) (2000 4%) 2.6509
2m 57s (- 53m 35s) (2100 5%) 2.7039
3m 2s (- 52m 35s) (2200 5%) 2.5078
3m 9s (- 52m 9s) (2300 5%) 2.6465
3m 16s (- 51m 39s) (2400 5%) 2.5935
3m 23s (- 51m 16s) (2500 6%) 2.6711
3m 30s (- 50m 53s) (2600 6%) 2.7231
3m 39s (- 50m 50s) (2700 6%) 2.6290
3m 47s (- 50m 44s) (2800 6%) 2.6869
3m 55s (- 

In [1258]:
evaluateRandomly(encoder1, decoder1)

> pei
= ꯄꯩ
< ꯄꯩ<EOS>

> yourakpada
= ꯌꯧꯔꯛꯄꯗ
< ꯌꯧꯔꯛꯄꯗ<EOS>

> khangnaribashi
= ꯈꯪꯅꯔꯤꯕꯁꯤ
< ꯈꯪꯍꯔꯥꯡꯅꯤꯁ<EOS>

> laagumna
= ꯂꯥꯒꯨꯝꯅ
< ꯂꯥꯒꯒꯝꯅ<EOS>

> sabitri
= ꯁꯥꯕꯤꯇ꯭ꯔꯤ
< ꯁꯥꯢꯕ꯭ꯔꯤ<EOS>

> kakchinglakpa
= ꯀꯛꯆꯤꯡꯂꯥꯛꯄ
< ꯀꯥꯢꯆꯪꯂꯛꯄꯄ<EOS>

> waaba
= ꯋꯥꯕ
< ꯋꯥꯕ<EOS>

> soitapa
= ꯁꯣꯢꯇꯄ
< ꯁꯤꯢꯇꯄ<EOS>

> apaamba
= ꯑꯄꯥꯝꯕ
< ꯑꯄꯥꯝꯕ<EOS>

> iriba
= ꯏꯔꯤꯕ
< ꯏꯔꯤꯕ<EOS>



In [1142]:
# pair = random.choice(pairs)

# print(pair[0])
# print(pair[1])

# with torch.no_grad():
#     input_tensor = tensorFromWord(input_lang, pair[0])
#     input_length = input_tensor.size()[0]
#     encoder1_hidden = encoder1.initHidden()

#     encoder1_outputs = torch.zeros(MAX_LENGTH, encoder1.hidden_size, device=device)

#     if encoder1.cell_type == 'lstm':
#         encoder1_cell = encoder1.initHidden()

#     for ei in range(input_length):
#         if encoder1.cell_type == 'lstm':
#             encoder1_output, encoder1_hidden, encoder1_cell = encoder1(input_tensor[ei], encoder1_hidden, encoder1_cell)
#         else:
#             encoder1_output, encoder1_hidden = encoder1(input_tensor[ei], encoder1_hidden)
            
#         encoder1_outputs[ei] += encoder1_output[0, 0]

#     decoder1_hidden = encoder1_hidden

#     if encoder1.cell_type == 'lstm':
#         decoder1_cell = encoder1_cell

#     decoded_chars = ""

#     # Predict the output using beam search
#     predicted_seq = decoder1.beam_search(encoder1_outputs, decoder1_hidden, 5, SOS_token, EOS_token)

#     # Convert the predicted sequence to a word
#     for char_idx in predicted_seq:
#         decoded_chars += output_lang.index2char[char_idx]

#     print(decoded_chars)

    



In [27]:
evalAccuracy(encoder1, decoder1, valid_pairs)

0.47822085889570554

In [28]:
evalAccuracy(encoder2, decoder2, valid_pairs)

0.4493865030674847

In [18]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'bayes-sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5, 10] 
        },
        'embedding_size': {
            'values': [64, 128, 256] 
        },
        'num_hidden_layers': {
            'values': [1, 2, 3]
        },
        'hidden_layer_size': {
            'values': [64, 128, 256] 
        },
        'cell_type': {
            'values': ['rnn', 'gru', 'lstm'] 
        },
        'learning_rate': {
            'values': [1e-2, 5e-3, 1e-3, 5e-4] 
        },
        'optimizer': {
            'values': ['SGD', 'Adam'] 
        },
        'dropout': {
            'values': [0.0, 0.2, 0.3] 
        },
        'beam_width': {
            'values': [1] #[1, 3, 5]
        }
    }
}


In [19]:
def hyperparameter_search():
    run = wandb.init(project='hyperparam-search-enc-dec', reinit=True)
    config = wandb.config
    wandb.run.name = f'hl_{config.num_hidden_layers}_sz_{config.hidden_layer_size}_ct_{config.cell_type}_lr_{config.learning_rate}_opt_{config.optimizer}_do_{config.dropout}_bw_{config.beam_width}_em_{config.embedding_size}'
    
    encoder = EncoderRNN(input_lang.n_chars, config.embedding_size, config.hidden_layer_size, config.num_hidden_layers, dropout=config.dropout, cell_type=config.cell_type).to(device)
    decoder = DecoderRNN(config.hidden_layer_size, config.embedding_size, output_lang.n_chars, config.num_hidden_layers, dropout=config.dropout, cell_type=config.cell_type).to(device)

    if config.optimizer == 'SGD':
        optimizer = optim.SGD
    elif config.optimizer == 'Adam':
        optimizer = optim.Adam

    trainBatch(encoder, decoder, config.epochs, optimizer=optimizer, beam_width=config.beam_width, print_every=100, learning_rate=config.learning_rate, log=True)
    run.finish()

In [20]:
sweep_id = wandb.sweep(sweep_configuration, project='hyperparam-search-seq2seq')
wandb.agent(sweep_id, hyperparameter_search, count=10)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: wpmuik2z
Sweep URL: https://wandb.ai/abdullah_010/hyperparam-search-seq2seq/sweeps/wpmuik2z


wandb: Agent Starting Run: rpddnz15 with config:
wandb: 	beam_width: 1
wandb: 	cell_type: rnn
wandb: 	dropout: 0
wandb: 	embedding_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.01
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: SGD
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cs20b001 (abdullah_010). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


0m 1s (- 13m 4s) (100 0%) 2.8626
0m 2s (- 12m 17s) (200 0%) 2.8268
0m 4s (- 11m 55s) (300 0%) 2.7690
0m 5s (- 11m 37s) (400 0%) 2.8099
0m 6s (- 11m 22s) (500 0%) 2.7305
0m 8s (- 11m 13s) (600 1%) 2.7081
0m 9s (- 11m 7s) (700 1%) 2.8027
0m 10s (- 11m 20s) (800 1%) 2.6996
0m 12s (- 11m 26s) (900 1%) 2.6892
0m 14s (- 11m 33s) (1000 1%) 2.7256
0m 15s (- 11m 40s) (1100 2%) 2.7105
0m 17s (- 11m 43s) (1200 2%) 2.7922
0m 18s (- 11m 44s) (1300 2%) 2.7667
0m 20s (- 11m 46s) (1400 2%) 2.6379
0m 21s (- 11m 45s) (1500 2%) 2.6406
0m 23s (- 11m 45s) (1600 3%) 2.6705
0m 24s (- 11m 43s) (1700 3%) 2.6209
0m 26s (- 11m 42s) (1800 3%) 2.7532
0m 27s (- 11m 42s) (1900 3%) 2.6397
0m 28s (- 11m 40s) (2000 3%) 2.5527
0m 30s (- 11m 39s) (2100 4%) 2.5570
0m 31s (- 11m 38s) (2200 4%) 2.6137
0m 33s (- 11m 37s) (2300 4%) 2.6377
0m 34s (- 11m 35s) (2400 4%) 2.5409
0m 36s (- 11m 35s) (2500 4%) 2.6220
0m 37s (- 11m 32s) (2600 5%) 2.6470
0m 39s (- 11m 32s) (2700 5%) 2.5764
0m 40s (- 11m 30s) (2800 5%) 2.5181
0m 42s (- 

epoch,▁▃▅▆█
train_accuracy,██▁▃▁
train_loss,▆▄▃▅▄▂▅▄▂▅▃▅▂▆▁▄▁▄▅▅▃▃▆▃▄▄█▂▄▃▂▆▃▅▅▃▆▄▁▂
val_accuracy,▁▁▁▁▁
epoch,5.0
train_accuracy,0.0
train_loss,2.5965
val_accuracy,0.0


...

0m 1s (- 22m 32s) (100 0%) 3.3241
0m 2s (- 19m 20s) (200 0%) 2.0862
0m 3s (- 18m 25s) (300 0%) 2.1634
0m 4s (- 17m 56s) (400 0%) 2.1603
0m 5s (- 17m 35s) (500 0%) 2.0379
0m 6s (- 17m 28s) (600 0%) 2.2348
0m 7s (- 17m 20s) (700 0%) 2.2017
0m 8s (- 17m 9s) (800 0%) 1.8904
0m 9s (- 17m 8s) (900 0%) 2.1510
0m 10s (- 17m 11s) (1000 0%) 1.9548
0m 11s (- 17m 25s) (1100 1%) 1.8055
0m 12s (- 17m 34s) (1200 1%) 2.0389
0m 13s (- 17m 39s) (1300 1%) 1.9953
0m 15s (- 17m 46s) (1400 1%) 2.1921
0m 16s (- 17m 51s) (1500 1%) 1.9731
0m 17s (- 17m 58s) (1600 1%) 2.2438
0m 18s (- 17m 58s) (1700 1%) 1.9178
0m 19s (- 18m 0s) (1800 1%) 2.1997
0m 20s (- 18m 4s) (1900 1%) 2.0709
0m 22s (- 18m 8s) (2000 1%) 2.2380
0m 23s (- 18m 10s) (2100 2%) 2.1034
0m 24s (- 18m 13s) (2200 2%) 2.3432
0m 25s (- 18m 16s) (2300 2%) 2.3171
0m 26s (- 18m 20s) (2400 2%) 2.4935
0m 28s (- 18m 27s) (2500 2%) 2.5217
0m 29s (- 18m 28s) (2600 2%) 2.4683
0m 30s (- 18m 31s) (2700 2%) 2.3839
0m 31s (- 18m 32s) (2800 2%) 2.4630
0m 33s (- 18m 3

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▁▁▁▁▁▂▄▆█
train_loss,█▇▆▄▄▆▅█▆▅▄▆▄▄▇▅▄▄▅▆▃▅▅▂▄▅▅▅▂▅▂▂▃█▅▃▄▆▁▁
val_accuracy,▁▁▁▁▁▁▂▄▆█
epoch,10.0
train_accuracy,0.22744
train_loss,0.54233
val_accuracy,0.24264


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: lwf5dmqo with config:
wandb: 	beam_width: 1
wandb: 	cell_type: lstm
wandb: 	dropout: 0.2
wandb: 	embedding_size: 256
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.0005
wandb: 	num_hidden_layers: 1
wandb: 	optimizer: SGD
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/home/abdullah/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


0m 1s (- 23m 58s) (100 0%) 3.1048
0m 2s (- 21m 36s) (200 0%) 2.1186
0m 3s (- 21m 15s) (300 0%) 2.2648
0m 5s (- 22m 5s) (400 0%) 2.1798
0m 6s (- 22m 58s) (500 0%) 2.2001
0m 8s (- 23m 20s) (600 0%) 2.0322
0m 10s (- 23m 49s) (700 0%) 2.0218
0m 11s (- 23m 44s) (800 0%) 1.7605
0m 12s (- 23m 46s) (900 0%) 2.2115
0m 14s (- 24m 45s) (1000 0%) 2.0076
0m 16s (- 25m 27s) (1100 1%) 1.7752
0m 18s (- 25m 26s) (1200 1%) 1.8212
0m 19s (- 25m 11s) (1300 1%) 1.8462
0m 21s (- 25m 30s) (1400 1%) 1.9123
0m 23s (- 25m 41s) (1500 1%) 1.7020
0m 25s (- 25m 55s) (1600 1%) 1.8542
0m 26s (- 25m 45s) (1700 1%) 1.9755
0m 28s (- 25m 45s) (1800 1%) 1.9048
0m 29s (- 25m 28s) (1900 1%) 1.8186
0m 31s (- 25m 28s) (2000 1%) 1.9949
0m 32s (- 25m 47s) (2100 2%) 1.7319
0m 34s (- 25m 45s) (2200 2%) 2.0009
0m 36s (- 25m 40s) (2300 2%) 2.1015
0m 37s (- 25m 40s) (2400 2%) 2.1534
0m 39s (- 25m 37s) (2500 2%) 2.1671
0m 41s (- 25m 56s) (2600 2%) 2.2147
0m 42s (- 25m 57s) (2700 2%) 2.2492
0m 44s (- 25m 54s) (2800 2%) 2.3993
0m 46s (